In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from cassandrautils import *


In [ ]:
tweets = getTwitter2DF()
weather = getWeatherDF()

tweets['Tweet timestamp'] = pd.to_datetime(tweets['tweet_date'], format="%d-%m-%Y %H:%M")
weather['forecast_timestamp'] = pd.to_datetime(weather['forecastdate'], format="%d-%m-%Y %H:%M")
weather['location'].replace(['United Kingdom of Great Britain and Northern Ireland'], ['United Kingdom'], inplace=True)

tweets = tweets[tweets['location'] == 'United Kingdom']
weather = weather[weather['location'] == 'United Kingdom']
print("Tweets from ", tweets['Tweet timestamp'].min(), " To ", tweets['Tweet timestamp'].max())
print("Weather from ", weather['forecast_timestamp'].min(), " To ", weather['forecast_timestamp'].max())

In [ ]:
tweets

In [ ]:
weather

In [ ]:
ns10min = 30*60*1000000000   # 30 minutes in nanoseconds 
tweets['Tweet timestamp 10mins'] = pd.to_datetime(((tweets['Tweet timestamp'].astype(np.int64) // ns10min + 1 ) * ns10min))
tweets['Tweet timestamp 10mins'] = pd.to_datetime(tweets['Tweet timestamp 10mins'], format="%d-%m-%Y %H:%M")

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(weather['forecast_timestamp'], weather['temp'])
plt.xlabel("Tweet time")
plt.ylabel("Temperature")
plt.title("Total weather data recorded")

In [ ]:
ww = weather[(weather['forecast_timestamp'] <= tweets['Tweet timestamp'].max())]
plt.figure(figsize=(20,6))
plt.plot(ww['forecast_timestamp'], ww['temp'])
plt.xlabel("Tweet time")
plt.ylabel("Temperature")
plt.title("Weather data within tweet window")

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(ww['forecast_timestamp'], ww['feels_like'])
plt.xlabel("Tweet time")
plt.ylabel("Feels like Temperature in (Kelvin)")

In [ ]:
value_counts = tweets.groupby(['Tweet timestamp 10mins'])['classification'].value_counts().rename_axis(['Tweet timestamp', 'classification']).reset_index(name='counts')
#value_counts.index.names = ['test', 'check']

value_counts.pivot("Tweet timestamp", "classification", "counts").plot(kind='bar', figsize=(20,10))
plt.xlabel("Tweet time")
plt.ylabel("Tweet Count")
plt.title("Plotting positive and negative tweet counts")
plt.show()


In [ ]:
def scoring(senti, c_val):
    if senti == "Positive":
        return c_val
    return c_val*(-1)


value_counts['adjusted'] = value_counts.apply(lambda x: scoring(x["classification"], x["counts"]), axis=1)
trend = value_counts.groupby(['Tweet timestamp'], as_index=False).agg({'adjusted' : "sum"})
plt.figure(figsize=(20,6))
plt.xlabel("Tweet time")
plt.ylabel("Net score")
plt.title("Plotting net tweet sentiments")
plt.plot(trend['Tweet timestamp'], trend['adjusted'])

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(ww['forecast_timestamp'], ww['feels_like'])
plt.xlabel("Tweet time")
plt.ylabel("Feels like Temperature in (Kelvin)")
plt.title("Plotting 'Feels like' temperature")